# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [4]:
import numpy as np
import pandas as pd
import requests #for making http requests
import math
from scipy import stats #makes it easy to calculate percentile score
import xlsxwriter
from secrets import IEX_CLOUD_API_TOKEN # we have the secrets.py file in this folder

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [5]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [10]:
api_url = api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
symbol = 'AAPL'
data = requests.get(api_url).json()
data


{'companyName': 'Apple Inc',
 'marketcap': 2256023683602,
 'week52high': 149.84,
 'week52low': 68.13,
 'week52highSplitAdjustOnly': 149.19,
 'week52lowSplitAdjustOnly': 67.02,
 'week52change': 0.997383275923321,
 'sharesOutstanding': 16920695219,
 'float': 0,
 'avg10Volume': 93535807,
 'avg30Volume': 104663422,
 'day200MovingAvg': 128.59,
 'day50MovingAvg': 127.42,
 'employees': 154223,
 'ttmEPS': 3.73,
 'ttmDividendRate': 0.8482235476302807,
 'dividendYield': 0.0065588569957963265,
 'nextDividendDate': '0',
 'exDividendDate': '2021-02-04',
 'nextEarningsDate': '2021-04-21',
 'peRatio': 35.16385316676587,
 'beta': 1.2989088665933375,
 'maxChangePercent': 50.5097057260024,
 'year5ChangePercent': 4.227013256006305,
 'year2ChangePercent': 1.73895620595751,
 'year1ChangePercent': 1.0121710465668328,
 'ytdChangePercent': -0.01669347656621425,
 'month6ChangePercent': 0.14206339616939218,
 'month3ChangePercent': -0.01151344447995719,
 'month1ChangePercent': 0.12489234119439692,
 'day30ChangeP

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [11]:
data['year1ChangePercent']

1.0121710465668328

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [16]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

for symbol in symbol_strings:
    print(symbol)


my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [71]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                ],index=my_columns
            ),ignore_index=True
        )
final_dataframe
    

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,135.90,0.722754,N/A
1,AAL,23.89,1.106614,N/A
2,AAP,185.70,0.781699,N/A
3,AAPL,136.49,0.993496,N/A
4,ABBV,108.00,0.432955,N/A
...,...,...,...,...
500,YUM,115.36,0.551117,N/A
501,ZBH,168.06,0.5134,N/A
502,ZBRA,515.40,1.621345,N/A
503,ZION,56.00,0.98046,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [72]:
final_dataframe.sort_values('One-Year Price Return',ascending=False,inplace=True) #inplace ensures that the original dataframe is modified
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,68.30,3.628248,N/A
1,179,FCX,34.48,3.285885,N/A
2,208,GPS,32.73,3.171944,N/A
3,272,KSS,63.40,2.571408,N/A
4,24,ALGN,581.59,2.142485,N/A
5,349,NWSA,27.77,2.09424,N/A
6,78,CARR,43.44,2.039752,N/A
7,441,TPR,43.77,1.983878,N/A
8,410,SIVB,493.23,1.955067,N/A
9,253,IVZ,26.08,1.947971,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [138]:
def portfolio_input():
    global portfolio_size
    flag=0
    while(flag == 0):
        try:
            portfolio_size = input('Enter the value of your portfolio: ')
            val = float(portfolio_size)
            flag=1
        except ValueError:
            print("That is not a number\nPlease try again")
    portfolio_size = val
    print(val)
portfolio_input()

Enter the value of your portfolio: 100000
100000.0


In [74]:
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,68.30,3.628248,N/A
1,179,FCX,34.48,3.285885,N/A
2,208,GPS,32.73,3.171944,N/A
3,272,KSS,63.40,2.571408,N/A
4,24,ALGN,581.59,2.142485,N/A
5,349,NWSA,27.77,2.09424,N/A
6,78,CARR,43.44,2.039752,N/A
7,441,TPR,43.77,1.983878,N/A
8,410,SIVB,493.23,1.955067,N/A
9,253,IVZ,26.08,1.947971,N/A


In [77]:
portfolio_size

10000000.0

In [80]:
position_size = float(portfolio_size)/len(final_dataframe.index)
print(position_size)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

200000.0


C:\Users\Attyuttam Saha\Anaconda3\envs\algorithmictrading\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,68.30,3.628248,2928
1,179,FCX,34.48,3.285885,5800
2,208,GPS,32.73,3.171944,6110
3,272,KSS,63.40,2.571408,3154
4,24,ALGN,581.59,2.142485,343
5,349,NWSA,27.77,2.09424,7202
6,78,CARR,43.44,2.039752,4604
7,441,TPR,43.77,1.983878,4569
8,410,SIVB,493.23,1.955067,405
9,253,IVZ,26.08,1.947971,7668


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [ ]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of shares to buy',
    'One-year price return',
    'One-year return percentile',
    'Six-month price return',
    'Six-month return percentile',
    'Three-month price return',
    'Three-month return percentile',
    'One-month price return',
    'One-month return percentile',
    'HQM score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)


for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                    
                ],index=hqm_columns
            ),ignore_index=True
        )
hqm_dataframe

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [ ]:
time_periods = [
    'One-year',
    'Six-month',
    'Three-month',
    'One-month'
]
#some values might come as "None", so we need to set them as 0, workaround from https://stackoverflow.com/questions/65174575/typeerror-not-supported-between-instances-of-nonetype-and-float
for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} price return'] == None:
            hqm_dataframe.loc[row, f'{time_period} price return'] = 0
for row in hqm_dataframe.index:
    for time_period in time_periods:
        
        change_col = f'{time_period} price return'
        percentile_col = f'{time_period} return percentile'
        
        hqm_dataframe.loc[row,percentile_col] = stats.percentileofscore(
           hqm_dataframe[change_col],
           hqm_dataframe.loc[row,change_col]
        )/100
hqm_dataframe

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [ ]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row,f'{time_period} return percentile'])
    hqm_dataframe.loc[row,'HQM score'] = mean(momentum_percentiles)
hqm_dataframe    

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [172]:
hqm_dataframe.sort_values('HQM score',ascending=False,inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of shares to buy,One-year price return,One-year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM score
0,LB,69.51,N/A,3.474022,100.0,1.078191,98.415842,0.449394,98.613861,0.223745,98.811881,98.960396
1,AMAT,144.90,N/A,1.863109,97.821782,1.24655,98.613861,0.473277,99.405941,0.240283,99.405941,98.811881
2,LRCX,678.00,N/A,1.652905,96.237624,0.861395,94.851485,0.332498,93.069307,0.285812,100.0,96.039604
3,KSS,63.47,N/A,2.192499,99.405941,1.993288,100.0,0.519395,99.60396,0.101571,84.158416,95.792079
4,MGM,43.13,N/A,1.900915,98.217822,0.980608,97.425743,0.337966,93.465347,0.11067,87.524752,94.158416
5,NUE,82.97,N/A,1.118661,87.722772,0.684014,88.118812,0.452036,98.811881,0.197827,98.415842,93.267327
6,KLAC,368.34,N/A,1.443086,94.257426,0.709321,90.09901,0.259522,87.722772,0.247864,99.80198,92.970297
7,CCL,29.70,N/A,1.374035,93.069307,0.906402,95.841584,0.436448,98.019802,0.100856,83.366337,92.574257
8,MHK,209.69,N/A,1.309261,91.881188,0.953683,97.029703,0.413416,97.029703,0.097242,81.386139,91.831683
9,ALK,75.66,N/A,1.327471,92.277228,0.918437,96.435644,0.434588,97.821782,0.095629,79.80198,91.584158


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [173]:
portfolio_input()

Enter the value of your portfolio: 1000000
1000000.0


In [174]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of shares to buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price']) 
hqm_dataframe

C:\Users\Attyuttam Saha\Anaconda3\envs\algorithmictrading\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of shares to buy,One-year price return,One-year return percentile,Six-month price return,Six-month return percentile,Three-month price return,Three-month return percentile,One-month price return,One-month return percentile,HQM score
0,LB,69.51,287,3.474022,100.0,1.078191,98.415842,0.449394,98.613861,0.223745,98.811881,98.960396
1,AMAT,144.90,138,1.863109,97.821782,1.24655,98.613861,0.473277,99.405941,0.240283,99.405941,98.811881
2,LRCX,678.00,29,1.652905,96.237624,0.861395,94.851485,0.332498,93.069307,0.285812,100.0,96.039604
3,KSS,63.47,315,2.192499,99.405941,1.993288,100.0,0.519395,99.60396,0.101571,84.158416,95.792079
4,MGM,43.13,463,1.900915,98.217822,0.980608,97.425743,0.337966,93.465347,0.11067,87.524752,94.158416
5,NUE,82.97,241,1.118661,87.722772,0.684014,88.118812,0.452036,98.811881,0.197827,98.415842,93.267327
6,KLAC,368.34,54,1.443086,94.257426,0.709321,90.09901,0.259522,87.722772,0.247864,99.80198,92.970297
7,CCL,29.70,673,1.374035,93.069307,0.906402,95.841584,0.436448,98.019802,0.100856,83.366337,92.574257
8,MHK,209.69,95,1.309261,91.881188,0.953683,97.029703,0.413416,97.029703,0.097242,81.386139,91.831683
9,ALK,75.66,264,1.327471,92.277228,0.918437,96.435644,0.434588,97.821782,0.095629,79.80198,91.584158


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [176]:
writer = pd.ExcelWriter('momentum_strategy.xlsx',engine='xlsxwriter')
hqm_dataframe.to_excel(writer,sheet_name='Momentum Strategy',index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [177]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [178]:
column_format = {
    'A':['Ticker',string_template],
    'B':['Price',dollar_template],
    'C':['Number of shares to buy',integer_template],
    'D':['One-year price return',percent_template],
    'E':['One-year return percentile',percent_template],
    'F':['Six-month price return',percent_template],
    'G':['Six-month return percentile',percent_template],
    'H':['Three-month price return',percent_template],
    'I':['Three-month return percentile',percent_template],
    'J':['One-month price return',percent_template],
    'K':['One-month return percentile',percent_template],
    'L':['HQM score',percent_template],
}

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [180]:
for column in column_format.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}',25,column_format[column][1])    
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_format[column][0],string_template)
writer.save()